<a href="https://colab.research.google.com/github/BrunoIannone/LABIAGI_grapesdetector/blob/main/IAGIr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) *Cloning* della repo di *Yolov5*, installazione dipendenze e *import* librerie


In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


# 2) *Download* del *dataset* da *Roboflow*


Specifico percorso del *dataset*

In [ ]:
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

Importiamo Roboflow e il modello (in questo caso *Yolov5*)

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

Codice generato da Roboflow per il *download* del dataset

In [ ]:
rf = Roboflow(api_key="olElazA0TsqKaw9m4wG7")
project = rf.workspace("iagi").project("iagi")
dataset = project.version(8).download("yolov5")

# 3) Addestramento di YOLOv5

Nel training specifichiamo i seguenti parametri:
- **img:** dimensione dell'immagine di input
- **batch:** dimensione dei *batch* delle immagini
- **epochs:** numero delle epoche per il *training*.
- **data:** la *location* del dataset è salvata in `dataset.location`
- **weights:** specifica un percorso ai pesi da cui iniziare l'apprendimento. In questo caso si utilizza un generico *checkpoint* di COCO.
- **cache:** *caching* delle immagini per velocizzare l'addestramento.

In [ ]:
!python train.py --img 640 --batch 64 --epochs 150 --data {dataset.location}/data.yaml  --cache 

# Valutazione performance del detector
*Training losses* e *performance* metriche vengono salvate in *Tensorboard* e in un *logfile*.


In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

#Verifica dei pesi
Con i pesi calcolati, si va a verificare le capacità del modello sul *test set*  contenuto in `test/images`.

Nel detecting specifichiamo i seguenti parametri:
- **weights:** specifica il percorso dei pesi da usare per il *detecting*.
- **img:** dimensione dell'immagine di input
- **source** specifica il percorso per le immagini del *test set*
- **line-thickness** dimensione (in pixel) delle *bounding box*
- **conf-thres** *confidence threshold*

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt  --img 640  --conf 0.1 --source {dataset.location}/test/images   --line-thickness 1  --conf-thres 0.51 

# 4) Verifica della *detection*

Infine, possiamo visualizzare i risultati calcolati sul *test set*

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# 5) Salvataggio dei pesi calcolati

Se i risultati sono soddisfacenti, è possibile scaricare i pesi calcolati

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')